# **Users By Average Session Time**

Calculate each user's average session time, where a session is defined as the time difference between a `page_load` and a `page_exit`. Assume each user has only one session per day. If there are multiple `page_load` or `page_exit` events on the same day, use only the latest `page_load` and the earliest `page_exit`. Only consider sessions where the `page_load` occurs before the `page_exit` on the same day. Output the `user_id` and their average session time.

### **Solution Walkthrough**
This Python solution uses the pandas library to calculate each user's average session duration. The logic mirrors SQL workflows by using `groupby`, `merge`, and `datetime` operations.

### **Understanding The Data**
The `facebook_web_log` DataFrame contains:

`user_id`: the identifier for each user

`timestamp`: a datetime value of the action

`action`: either `'page_load'` or `'page_exit'`

Each row represents a single user action. One valid session per user per day is the goal.

### **The Problem Statement**
We need to:

Create a "day" column to group data by calendar day.

Extract the latest `page_load` and the earliest `page_exit` per user/day.

Calculate the difference between those two timestamps in seconds.

Aggregate the average session duration for each user.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
facebook_web_log['timestamp'] = pd.to_datetime(facebook_web_log['timestamp'])

# create a date column
facebook_web_log['dates'] = facebook_web_log['timestamp'].dt.date

# rank the actions based on dates
facebook_web_log = facebook_web_log.sort_values('timestamp')
facebook_web_log['action_rank'] = facebook_web_log.groupby(['user_id','dates', 'action']).cumcount()+1

# get page loads
page_loads = facebook_web_log[facebook_web_log['action'] == 'page_load']
last_page_loads = page_loads.groupby(['user_id', 'dates'])['timestamp'].max().reset_index()

# page_exits
page_exits = facebook_web_log[facebook_web_log['action'] == 'page_exit']
first_page_exits = page_exits.groupby(['user_id', 'dates'])['timestamp'].min().reset_index()

# merge loads and exits
merged_df = pd.merge(last_page_loads, first_page_exits, on=['user_id', 'dates'], suffixes=('_loads', '_exits'))

# get session time
merged_df['session_time'] = merged_df['timestamp_exits'] - merged_df['timestamp_loads']
merged_df = merged_df[['user_id', 'dates', 'session_time']].groupby('user_id')['session_time'].mean().reset_index()

# get average session_time
merged_df = merged_df.rename(columns={"session_time": "avg_session_time"})
merged_df